라이브러리 및 데이터불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
data = pd.read_csv('Korea Income and Welfare.csv')
data

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,11600.0,6,1,1967,5,1,1,874,1,
92853,98000701,2015,11,5,8327.0,6,1,1967,5,1,1,874,1,
92854,98000701,2016,12,5,7931.0,6,1,1967,5,1,1,874,1,
92855,98000701,2017,13,5,8802.0,5,1,1967,5,1,1,874,1,


사용하지 않을 열 제거

In [2]:
data = data.drop(['id','occupation','company_size','reason_none_worker','wave'],axis=1)

# 가장 최근 년도 검색

In [3]:
year_max = max(data['year'])
year_max

2018

In [4]:
data = data[data['year']==2018]

데이터가 얼마나 줄어들었는지 확인

In [5]:
len(data)

6331

In [6]:
# 다쓰고 필요없는 열 추가 제거
data = data.drop(['year'], axis=1)
data

,region,income,family_member,gender,year_born,education_level,marriage,religion
21,1,2330.0,1,2,1945,4,2,1
35,1,815.0,1,1,1948,3,2,2
49,1,2116.0,1,1,1942,7,3,1
64,1,5008.0,5,1,1962,6,1,1
88,2,1332.0,1,2,1940,3,2,1
...,...,...,...,...,...,...,...,...
92817,6,557.0,1,2,1947,2,2,2
92825,6,1948.0,2,1,1933,3,1,2
92833,6,866.0,1,2,1934,3,2,1
92841,6,4118.0,2,1,1956,5,1,1


In [7]:
# 중복된 행 삭제
data.drop_duplicates()
len(data) # 중복된 행이 없는 것을 확인

6331

In [8]:
# 누락 데이터 찾기
data.isnull().sum()

region             0
income             0
family_member      0
gender             0
year_born          0
education_level    0
marriage           0
religion           0
dtype: int64

In [9]:
# 통계값 확인하기 
data.describe()

,region,income,family_member,gender,year_born,education_level,marriage,religion
count,6331.000000,6331.000000,6331.000000,6331.000000,6331.000000,6331.000000,6331.000000,6331.000000
mean,3.788817,4273.601279,2.277365,1.342916,1955.559627,4.591692,1.766546,1.521245
std,2.043394,4509.175617,1.240378,0.474721,16.390786,1.674147,1.135312,0.499588
min,1.000000,-46897.000000,1.000000,1.000000,1922.000000,2.000000,1.000000,1.000000
25%,2.000000,1418.500000,1.000000,1.000000,1942.000000,3.000000,1.000000,1.000000
50%,3.000000,2987.000000,2.000000,1.000000,1954.000000,5.000000,1.000000,2.000000
75%,6.000000,5984.500000,3.000000,2.000000,1969.000000,6.000000,2.000000,2.000000
max,7.000000,170204.000000,9.000000,2.000000,1999.000000,9.000000,6.000000,2.000000


In [10]:
normalization_data = (data - data.min())/data.max()
print(data.corr())
print(normalization_data.corr())

                   region    income  family_member    gender  year_born  \
region           1.000000 -0.102950      -0.052242  0.024832  -0.096325   
income          -0.102950  1.000000       0.546047 -0.350003   0.415581   
family_member   -0.052242  0.546047       1.000000 -0.513012   0.452891   
gender           0.024832 -0.350003      -0.513012  1.000000  -0.296624   
year_born       -0.096325  0.415581       0.452891 -0.296624   1.000000   
education_level -0.139181  0.471028       0.447023 -0.434835   0.659478   
marriage        -0.041684 -0.245391      -0.480486  0.386279   0.171251   
religion         0.086955  0.042024       0.072579 -0.145626   0.186195   

                 education_level  marriage  religion  
region                 -0.139181 -0.041684  0.086955  
income                  0.471028 -0.245391  0.042024  
family_member           0.447023 -0.480486  0.072579  
gender                 -0.434835  0.386279 -0.145626  
year_born               0.659478  0.171251  0.186

In [11]:
# 이 사이에 거리 측정? 

In [12]:
# 값 치환 (보기 편하게 하기 위해서)
data['region'] = data['region'].replace([1,2,3,4,5,6,7],['Seoul','Kyeong-gi','Kyoung-nam','Kyoung-buk','Chung-nam','Gang-won &. Chung-buk', 'Jeolla & Jeju'])
data['education_level'] = data['education_level'].replace([1,2,3,4,5,6,7,8,9],['no education','no education','elementary','middle school','high school','college','university degree','MA','doctoral degree'])
data['gender'] = data['gender'].replace([1,2],['male','female'])
data['religion'] = data['religion'].replace([1,2],['have','do_not'])

In [13]:
#변경된 데이터 확인
data

,region,income,family_member,gender,year_born,education_level,marriage,religion
21,Seoul,2330.0,1,female,1945,middle school,2,have
35,Seoul,815.0,1,male,1948,elementary,2,do_not
49,Seoul,2116.0,1,male,1942,university degree,3,have
64,Seoul,5008.0,5,male,1962,college,1,have
88,Kyeong-gi,1332.0,1,female,1940,elementary,2,have
...,...,...,...,...,...,...,...,...
92817,Gang-won &. Chung-buk,557.0,1,female,1947,no education,2,do_not
92825,Gang-won &. Chung-buk,1948.0,2,male,1933,elementary,1,do_not
92833,Gang-won &. Chung-buk,866.0,1,female,1934,elementary,2,have
92841,Gang-won &. Chung-buk,4118.0,2,male,1956,high school,1,have


In [14]:
# 통계값 확인하기 
data.describe()

,income,family_member,year_born,marriage
count,6331.000000,6331.000000,6331.000000,6331.000000
mean,4273.601279,2.277365,1955.559627,1.766546
std,4509.175617,1.240378,16.390786,1.135312
min,-46897.000000,1.000000,1922.000000,1.000000
25%,1418.500000,1.000000,1942.000000,1.000000
50%,2987.000000,2.000000,1954.000000,1.000000
75%,5984.500000,3.000000,1969.000000,2.000000
max,170204.000000,9.000000,1999.000000,6.000000


In [15]:
data.cov()

C:\Users\cjf46\AppData\Local\Temp\ipykernel_16308\4020818503.py:1: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.cov()


,income,family_member,year_born,marriage
income,2.033266e+07,3054.085285,30715.132996,-1256.237727
family_member,3.054085e+03,1.538538,9.207629,-0.676628
year_born,3.071513e+04,9.207629,268.657858,3.186750
marriage,-1.256238e+03,-0.676628,3.186750,1.288934


In [17]:
# 지역별 평균 소득 확인
data.groupby(['region'])['income'].mean()

region
Chung-nam                4817.839701
Gang-won &. Chung-buk    3754.438921
Jeolla & Jeju            3553.840433
Kyeong-gi                5095.418098
Kyoung-buk               3587.831835
Kyoung-nam               4022.111470
Seoul                    4870.470314
Name: income, dtype: float64

In [18]:
# onhot encoding
dummy1 = pd.get_dummies(data[['religion']], drop_first=True)
dummy2 = pd.get_dummies(data[['region']], drop_first=True)
dummy3 = pd.get_dummies(data[['education_level']], drop_first=True)
dummy4 = pd.get_dummies(data[['gender']], drop_first=True)
data = pd.concat([data, dummy1, dummy2, dummy3, dummy4], axis=1)
data =data.drop(['religion','region','education_level','gender'],axis=1)

In [19]:
data

,income,family_member,year_born,marriage,religion_have,region_Gang-won &. Chung-buk,region_Jeolla & Jeju,region_Kyeong-gi,region_Kyoung-buk,region_Kyoung-nam,region_Seoul,education_level_college,education_level_doctoral degree,education_level_elementary,education_level_high school,education_level_middle school,education_level_no education,education_level_university degree,gender_male
21,2330.0,1,1945,2,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
35,815.0,1,1948,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
49,2116.0,1,1942,3,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1
64,5008.0,5,1962,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1
88,1332.0,1,1940,2,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92817,557.0,1,1947,2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
92825,1948.0,2,1933,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
92833,866.0,1,1934,2,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0
92841,4118.0,2,1956,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1


In [20]:
normalization_data = (data - data.min())/data.max()
normalization_data

,income,family_member,year_born,marriage,religion_have,region_Gang-won &. Chung-buk,region_Jeolla & Jeju,region_Kyeong-gi,region_Kyoung-buk,region_Kyoung-nam,region_Seoul,education_level_college,education_level_doctoral degree,education_level_elementary,education_level_high school,education_level_middle school,education_level_no education,education_level_university degree,gender_male
21,0.289224,0.000000,0.011506,0.166667,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
35,0.280322,0.000000,0.013007,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
49,0.287966,0.000000,0.010005,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
64,0.304958,0.444444,0.020010,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
88,0.283360,0.000000,0.009005,0.166667,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92817,0.278807,0.000000,0.012506,0.166667,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
92825,0.286979,0.111111,0.005503,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
92833,0.280622,0.000000,0.006003,0.166667,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
92841,0.299729,0.111111,0.017009,0.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [21]:
data.corr()

,income,family_member,year_born,marriage,religion_have,region_Gang-won &. Chung-buk,region_Jeolla & Jeju,region_Kyeong-gi,region_Kyoung-buk,region_Kyoung-nam,region_Seoul,education_level_college,education_level_doctoral degree,education_level_elementary,education_level_high school,education_level_middle school,education_level_no education,education_level_university degree,gender_male
income,1.000000,0.546047,0.415581,-0.245391,-0.042024,-0.034408,-0.073525,0.092832,-0.058792,-0.025803,0.052449,0.109549,0.088896,-0.263960,0.109292,-0.105968,-0.219453,0.300878,0.350003
family_member,0.546047,1.000000,0.452891,-0.480486,-0.072579,-0.020868,-0.019212,0.087015,-0.055392,-0.031262,0.017011,0.167982,0.010045,-0.277756,0.188941,-0.082162,-0.261598,0.219566,0.513012
year_born,0.415581,0.452891,1.000000,0.171251,-0.186195,-0.032828,-0.046848,0.102622,-0.079713,-0.031780,0.057475,0.309816,0.050155,-0.376963,0.181646,-0.151897,-0.354104,0.365686,0.296624
marriage,-0.245391,-0.480486,0.171251,1.000000,-0.058881,-0.010061,-0.016233,-0.014648,-0.025597,-0.006742,0.074165,0.035151,0.015803,0.010787,-0.016341,-0.026572,0.040966,-0.019290,-0.386279
religion_have,-0.042024,-0.072579,-0.186195,-0.058881,1.000000,-0.022446,-0.096079,-0.011960,0.031012,0.093524,0.022851,-0.073901,0.002835,0.076262,-0.036825,0.035786,0.013239,-0.048492,-0.145626
region_Gang-won &. Chung-buk,-0.034408,-0.020868,-0.032828,-0.010061,-0.022446,1.000000,-0.137636,-0.152197,-0.115511,-0.138237,-0.118398,0.002682,0.006114,0.038760,-0.025103,-0.011321,0.015966,-0.012097,0.019375
region_Jeolla & Jeju,-0.073525,-0.019212,-0.046848,-0.016233,-0.096079,-0.137636,1.000000,-0.234582,-0.178038,-0.213066,-0.182488,-0.002815,0.000624,0.047146,-0.026045,-0.007312,0.046328,-0.041457,-0.029818
region_Kyeong-gi,0.092832,0.087015,0.102622,-0.014648,-0.011960,-0.152197,-0.234582,1.000000,-0.196874,-0.235608,-0.201794,0.032051,0.001370,-0.062454,0.060268,-0.020208,-0.067583,0.037387,0.056085
region_Kyoung-buk,-0.058792,-0.055392,-0.079713,-0.025597,0.031012,-0.115511,-0.178038,-0.196874,1.000000,-0.178817,-0.153153,-0.006230,-0.011607,0.063013,-0.030848,0.005221,0.033519,-0.047052,-0.037382
region_Kyoung-nam,-0.025803,-0.031262,-0.031780,-0.006742,0.093524,-0.138237,-0.213066,-0.235608,-0.178817,1.000000,-0.183286,-0.017938,-0.012092,-0.005253,-0.000070,0.022802,0.039590,-0.024557,-0.024718


In [22]:
data.cov()

,income,family_member,year_born,marriage,religion_have,region_Gang-won &. Chung-buk,region_Jeolla & Jeju,region_Kyeong-gi,region_Kyoung-buk,region_Kyoung-nam,region_Seoul,education_level_college,education_level_doctoral degree,education_level_elementary,education_level_high school,education_level_middle school,education_level_no education,education_level_university degree,gender_male
income,2.033266e+07,3054.085285,30715.132996,-1256.237727,-94.668875,-42.566392,-125.986575,169.296861,-89.160861,-44.338488,80.996920,129.889036,26.600816,-498.606072,219.434470,-163.966178,-298.735327,494.022184,749.217406
family_member,3.054085e+03,1.538538,9.207629,-0.676628,-0.044975,-0.007102,-0.009055,0.043652,-0.023108,-0.014777,0.007226,0.054788,0.000827,-0.144324,0.104351,-0.034971,-0.097957,0.099170,0.302079
year_born,3.071513e+04,9.207629,268.657858,3.186750,-1.524681,-0.147622,-0.291796,0.680292,-0.439427,-0.198506,0.322635,1.335276,0.054555,-2.588339,1.325694,-0.854338,-1.752177,2.182561,2.308049
marriage,-1.256238e+03,-0.676628,3.186750,1.288934,-0.033396,-0.003134,-0.007004,-0.006726,-0.009774,-0.002917,0.028837,0.010494,0.001191,0.005130,-0.008261,-0.010352,0.014041,-0.007975,-0.208188
religion_have,-9.466887e+01,-0.044975,-1.524681,-0.033396,0.249588,-0.003076,-0.018240,-0.002417,0.005211,0.017806,0.003910,-0.009708,0.000094,0.015960,-0.008192,0.006135,0.001997,-0.008822,-0.034537
region_Gang-won &. Chung-buk,-4.256639e+01,-0.007102,-0.147622,-0.003134,-0.003076,0.075269,-0.014349,-0.016888,-0.010658,-0.014453,-0.011125,0.000193,0.000111,0.004455,-0.003067,-0.001066,0.001322,-0.001208,0.002523
region_Jeolla & Jeju,-1.259866e+02,-0.009055,-0.291796,-0.007004,-0.018240,-0.014349,0.144406,-0.036053,-0.022754,-0.030855,-0.023750,-0.000281,0.000016,0.007505,-0.004407,-0.000953,0.005315,-0.005736,-0.005379
region_Kyeong-gi,1.692969e+02,0.043652,0.680292,-0.006726,-0.002417,-0.016888,-0.036053,0.163573,-0.026779,-0.036313,-0.027951,0.003409,0.000037,-0.010581,0.010853,-0.002805,-0.008252,0.005506,0.010768
region_Kyoung-buk,-8.916086e+01,-0.023108,-0.439427,-0.009774,0.005211,-0.010658,-0.022754,-0.026779,0.113114,-0.022919,-0.017641,-0.000551,-0.000259,0.008878,-0.004620,0.000603,0.003403,-0.005762,-0.005968
region_Kyoung-nam,-4.433849e+01,-0.014777,-0.198506,-0.002917,0.017806,-0.014453,-0.030855,-0.036313,-0.022919,0.145225,-0.023921,-0.001798,-0.000306,-0.000839,-0.000012,0.002982,0.004555,-0.003408,-0.004472


In [23]:
data

,income,family_member,year_born,marriage,religion_have,region_Gang-won &. Chung-buk,region_Jeolla & Jeju,region_Kyeong-gi,region_Kyoung-buk,region_Kyoung-nam,region_Seoul,education_level_college,education_level_doctoral degree,education_level_elementary,education_level_high school,education_level_middle school,education_level_no education,education_level_university degree,gender_male
21,2330.0,1,1945,2,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
35,815.0,1,1948,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
49,2116.0,1,1942,3,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1
64,5008.0,5,1962,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1
88,1332.0,1,1940,2,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92817,557.0,1,1947,2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
92825,1948.0,2,1933,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
92833,866.0,1,1934,2,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0
92841,4118.0,2,1956,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1


In [24]:
# 첫번째 집과 가장 거리가 가까운 샘플 번호 
# 정규화를 한 이유 income의 영향이 너무 커서 줄이기 위해 

distance_vector1 = np.linalg.norm(normalization_data.iloc[0] - normalization_data.iloc[1:], axis=1, ord=1)
num1 = np.argmin(distance_vector1)
distance_vector2 = np.linalg.norm(data.iloc[0] - data.iloc[1:], axis=1, ord=1)
num2 = np.argmin(distance_vector2)

In [25]:
# 정규화전
data.iloc[0] , data.iloc[num2+1]

(income                               2330.0
 family_member                           1.0
 year_born                            1945.0
 marriage                                2.0
 religion_have                           1.0
 region_Gang-won &. Chung-buk            0.0
 region_Jeolla & Jeju                    0.0
 region_Kyeong-gi                        0.0
 region_Kyoung-buk                       0.0
 region_Kyoung-nam                       0.0
 region_Seoul                            1.0
 education_level_college                 0.0
 education_level_doctoral degree         0.0
 education_level_elementary              0.0
 education_level_high school             0.0
 education_level_middle school           1.0
 education_level_no education            0.0
 education_level_university degree       0.0
 gender_male                             0.0
 Name: 21, dtype: float64,
 income                               2330.0
 family_member                           1.0
 year_born                  

In [26]:
# 정규화 후
data.iloc[0] , data.iloc[num1+1]

(income                               2330.0
 family_member                           1.0
 year_born                            1945.0
 marriage                                2.0
 religion_have                           1.0
 region_Gang-won &. Chung-buk            0.0
 region_Jeolla & Jeju                    0.0
 region_Kyeong-gi                        0.0
 region_Kyoung-buk                       0.0
 region_Kyoung-nam                       0.0
 region_Seoul                            1.0
 education_level_college                 0.0
 education_level_doctoral degree         0.0
 education_level_elementary              0.0
 education_level_high school             0.0
 education_level_middle school           1.0
 education_level_no education            0.0
 education_level_university degree       0.0
 gender_male                             0.0
 Name: 21, dtype: float64,
 income                               2053.0
 family_member                           1.0
 year_born                  

In [33]:
# 정규화o 코사인데이터
cosine_similarity(np.array(normalization_data.iloc[0]).reshape(1,-1), np.array(normalization_data.iloc[num1]).reshape(1,-1))

array([[0.34687961]])

In [34]:
# 정규화x 코사인데이터
cosine_similarity(np.array(data.iloc[0]).reshape(1,-1), np.array(data.iloc[num2]).reshape(1,-1))

array([[0.99661552]])